In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import theano as tt
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
train = pd.read_csv('/home/jordan/Data/ELo/elo-merchant-category-recommendation/train.csv')
test = pd.read_csv('/home/jordan/Data/ELo/elo-merchant-category-recommendation/test.csv')
merchants = pd.read_csv('/home/jordan/Data/ELo/elo-merchant-category-recommendation/merchants.csv')
new_merchant_t = pd.read_csv('/home/jordan/Data/ELo/elo-merchant-category-recommendation/new_merchant_transactions.csv')
his_trans = pd.read_csv('/home/jordan/Data/ELo/elo-merchant-category-recommendation/historical_transactions.csv')

In [3]:
train['first_active_month'] = pd.to_datetime(train['first_active_month']).apply(lambda x: x.strftime('%Y-%m'))
test['first_active_month'] = test['first_active_month'].fillna('2017-09')
test['first_active_month'] = pd.to_datetime(test['first_active_month']).apply(lambda x: x.strftime('%Y-%m'))   

In [4]:
train['first_active_month'] = pd.to_datetime(train['first_active_month']).apply(lambda x: x.strftime('%Y-%m'))
test['first_active_month'] = test['first_active_month'].fillna('2017-09')
test['first_active_month'] = pd.to_datetime(test['first_active_month']).apply(lambda x: x.strftime('%Y-%m')) 

In [5]:
c_his = his_trans.groupby("card_id")
c_his = c_his["purchase_amount"].agg(['size', 'sum']).reset_index()
c_his.columns = ["card_id","purchase_amount_size", "purchase_amount_sum"]
train = pd.merge(train, c_his, on="card_id", how="left")
test = pd.merge(test, c_his, on = "card_id", how = "left")

In [6]:
his_d = pd.get_dummies(his_trans, columns = ['category_1'], drop_first=True)

In [7]:
c_his = his_d.groupby('card_id')
c_his = c_his['category_1_Y'].size().reset_index()
c_his.columns = ['card_id', 'category_1_y_size']
train = pd.merge(train, c_his, on = 'card_id', how = 'left')
test = pd.merge(test, c_his, on = 'card_id', how = 'left')

In [8]:
c_his = his_trans.groupby('card_id')
c_his = c_his['merchant_id'].nunique().reset_index()
c_his.columns = ['card_id', 'merchant_id_nunique']
train = pd.merge(train, c_his, on = 'card_id', how = 'left')
test = pd.merge(test, c_his, on = 'card_id', how = 'left')

In [9]:
c_his = his_trans.groupby("card_id")
c_his = c_his["month_lag"].max().reset_index()
c_his.columns = ["card_id","month_lag_recent"]
train = pd.merge(train, c_his, on="card_id", how="left")

In [10]:
from datetime import datetime, timedelta

In [11]:
his_trans['month_lag_month'] = pd.to_datetime(his_trans['purchase_date']).apply(lambda x: x.strftime('%Y-%m'))

In [12]:
c_his = his_trans.groupby("card_id")
c_his = c_his["month_lag_month"].max().reset_index()
c_his.columns = ["card_id","month_lag_month_recent"]
train = pd.merge(train, c_his, on="card_id", how="left")
test = pd.merge(test, c_his, on = 'card_id', how = 'left')

In [13]:
train['purchase_period'] = pd.to_datetime(train['month_lag_month_recent'])-pd.to_datetime(train['first_active_month'])
test['purchase_period'] = pd.to_datetime(test['month_lag_month_recent'])-pd.to_datetime(test['first_active_month'])

In [14]:
train.purchase_period = train['purchase_period'].dt.days
test.purchase_period = test['purchase_period'].dt.days

In [15]:
colnames = ['feature_1', 'feature_2', 'feature_3', 'purchase_amount_size', 'purchase_amount_sum'
            , 'merchant_id_nunique', 'purchase_period']

target = train['target']
x = train[colnames]

X_train, X_test, Y_train, Y_test = train_test_split(x, target, test_size=0.2)

In [28]:
X_test.dtypes

feature_1                 int64
feature_2                 int64
feature_3                 int64
purchase_amount_size      int64
purchase_amount_sum     float64
merchant_id_nunique       int64
purchase_period           int64
dtype: object

In [65]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(X_train, label = Y_train)
dtest = xgb.DMatrix(X_test, label = Y_test)
# specify parameters via map
param = {'max_depth':10, 'eta':1, 'silent':1, 'objective':'reg:linear' }
param['eval_metric'] = 'mae'
num_round = 5
bst = xgb.train(param, dtrain, num_round)
# make prediction
#preds = bst.predict(test[colnames])

In [66]:
preds = bst.predict(xgb.DMatrix(test[colnames]))

In [67]:
pred = pd.DataFrame(preds)

In [68]:
xgb_submission.head()

,card_id,target
0,C_ID_0ab67a22ab,-1.406513
1,C_ID_130fd0cbdd,2.269229
2,C_ID_b709037bc5,0.273058
3,C_ID_d27d835a9f,-0.176821
4,C_ID_2b5e3df5c2,-0.995465


In [69]:
xgb_submission = test.join(pred, how = 'outer')
xgb_submission = xgb_submission[['card_id', 0]]
xgb_submission.columns = ['card_id', 'target']


In [70]:
xgb_submission.to_csv('/home/jordan/Data/ELo/elo-merchant-category-recommendation/xgb_submission', sep = ',', index = False)